## CS431/631 Data Intensive Distributed Analytics
### Fall 2021 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Ze Sheng
* **ID:** 20848004

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

and then create a `SparkContext`:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [3]:
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    #### Your code for Question 1.1 should go here
    def _edge_mapper(line):
        adjs = len(line.split()) - 1
        return (adjs + abs(adjs)) // 2

    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    serNum = lines.map(lambda x: x != '').reduce(lambda x, y: x + y)
    edgeNum = lines.map(_edge_mapper).reduce(lambda x, y: x + y)
    return serNum, edgeNum

    
def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """
    #### Your code for Question 1.2 should go here
    def _outDeg_mapper(line):
        lth = len(line.split())
        return (lth - 1, 1) if lth else (-1, 0)

    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    ret = lines.map(_outDeg_mapper).reduceByKey(lambda x, y: x + y).collectAsMap()
    ret.pop(-1) if -1 in ret else None
    return ret


def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """
    #### Your code for Question 1.3 should go here
    def _inDeg_mapper(line):
        nodes = line.split()
        return [(int(node), 1) for node in nodes[1:]]

    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    ret = lines.flatMap(_inDeg_mapper).reduceByKey(lambda x, y: x + y).map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y).collectAsMap()
    return ret



---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [7]:
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 2 here
    def _init_mapper(line):
        nodes = list(map(int, line.split()))
        outDeg = len(nodes) - 1
        nodes.append(-1)
        key = nodes[0]
        return (key, (0, nodes[1:-1])) if key != source_node_id else (key, (1, nodes[1:-1]))

    def _iter_mapper(data):
        node, (mass, adjList) = data
        emit = []
        if adjList:
            putBack = mass * jump_factor
            emit.append((node, (0.0, adjList)))
            propagation = (mass - putBack) / len(adjList)
            for adj in adjList:
                emit.append((adj, (propagation, [])))
        else:
            putBack = mass
        emit.append((source_node_id, (putBack, [])))
        return emit

    def _iter_reducer(x, y):
        return (x[0] + y[0], x[1] + y[1])
    

    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    graph = lines.map(_init_mapper)
    for i in range(num_iterations):
        graph = graph.flatMap(_iter_mapper).reduceByKey(_iter_reducer)

    return graph.map(lambda x: (x[0], x[1][0])).top(10, lambda x: x[1])



---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [9]:
from operator import add, or_

def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 3 here
    def _get_num_nodes(lines):
        return lines.map(lambda x: x != '').reduce(add)

    def _init_mapper(line):
        nodes = list(map(int, line.split()))
        outDeg = len(nodes) - 1
        nodes.append(-1)
        key = nodes[0]
        return (key, (0, 0, nodes[1:-1])) if key != source_node_id else (key, (1, 0, nodes[1:-1]))

    def _iter_mapper(data):
        node, (mass, regMass, adjList) = data
        emit = [(node, (0.0, mass, adjList))]
        if adjList:
            putBack = mass * jump_factor
            propagation = (mass - putBack) / len(adjList)
            for adj in adjList:
                emit.append((adj, (propagation, 0.0, [])))
        else:
            putBack = mass
        emit.append((source_node_id, (putBack, 0.0, [])))
        return emit

    def _iter_reducer(x, y):
        return (x[0] + y[0], x[1] + y[1], x[2] + y[2])

    def _trigger_mapper(data):
        return abs(data[1][0] - data[1][1]) > threshold
    

    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    lines.cache()
    threshold = 0.5 / _get_num_nodes(lines)
    graph = lines.map(_init_mapper)
    numPart = graph.getNumPartitions()
    trigger = True
    while trigger:
        graph = graph.flatMap(_iter_mapper).reduceByKey(_iter_reducer)
        # For Question 4, here is the place to insert `graph.cache()`. 
        # However, for this circumstance, 
        # using `cache()` performs much worse than using 'partitionBy()'.
        # (One reason is that the job automatically optimized by Spark
        #  runs cheaper than the accumulated overhead of `cache()`, 
        #  such as maintaining LRU for the RDDs)
        graph.cache()
        trigger = graph.map(_trigger_mapper).reduce(or_)
        # The alternative approch without `cache()`
        # trigger = graph.partitionBy(numPart).map(_trigger_mapper).reduce(or_)

    return graph.map(lambda x: (x[0], x[1][0])).top(10, lambda x: x[1])



---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

**Cache the RDDs reused (followed by different chains of transformations) in between two actions.**

**In other word, if several actual jobs share a common chain of transformations, then it is reasonable to consider caching at the branch point.**

P.S. For my implementation of Question 3, we `reduce` the RDD `graph` to the scalar `trigger` in each iteration, which branches out from the long chain of trasforming the `graph`. When it executes the action `top` in the end, it reuses the RDD cached in the last iteration.  




---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.